# Try to find which molecular orbitals are used in calculating MI


In [6]:
import numpy as np
from sympy import symbols, Matrix, simplify
import matplotlib.pyplot as plt
from scipy.linalg import fractional_matrix_power
from scipy.optimize import brentq
import matplotlib.ticker as ticker
import sys
sys.path.append('/workspaces/bachelor-project-bridging-profile-Mattice-Criel/projects/Project-MatticeCriel/Benzene/rhf')
import plottingsystem as pts
import sys
sys.path.append('/workspaces/bachelor-project-bridging-profile-Mattice-Criel/projects/Project-MatticeCriel/Benzene/Huckel')
from class_huckel import huckel

In [25]:
benzene_molecule = '''
  C    1.2116068    0.6995215    0.0000000
  C    1.2116068   -0.6995215    0.0000000
  C   -0.0000000   -1.3990430   -0.0000000
  C   -1.2116068   -0.6995215   -0.0000000
  C   -1.2116068    0.6995215   -0.0000000
  C    0.0000000    1.3990430    0.0000000
  H    2.1489398    1.2406910    0.0000000
  H    2.1489398   -1.2406910    0.0000000
  H   -0.0000000   -2.4813820   -0.0000000
  H   -2.1489398   -1.2406910   -0.0000000
  H   -2.1489398    1.2406910   -0.0000000
  H    0.0000000    2.4813820    0.0000000
'''
extra_bounds = [(0,5)]
coordinates = pts.taking_x_and_y_coordinate(pts.taking_carbon_coordinates(benzene_molecule))
print(coordinates)
coordinates = np.vstack((coordinates[-1], coordinates[[0, 1, 2, 3, 4]]))
print(coordinates)

[[ 1.2116068  0.6995215]
 [ 1.2116068 -0.6995215]
 [-0.        -1.399043 ]
 [-1.2116068 -0.6995215]
 [-1.2116068  0.6995215]
 [ 0.         1.399043 ]]
[[ 0.         1.399043 ]
 [ 1.2116068  0.6995215]
 [ 1.2116068 -0.6995215]
 [-0.        -1.399043 ]
 [-1.2116068 -0.6995215]
 [-1.2116068  0.6995215]]


In [4]:
benzene = huckel(6, 0, -3)
benzene.create_matrix()

H = benzene.huckel_matrix
print(H)

[[ 0. -3.  0.  0.  0. -3.]
 [-3.  0. -3.  0.  0.  0.]
 [ 0. -3.  0. -3.  0.  0.]
 [ 0.  0. -3.  0. -3.  0.]
 [ 0.  0.  0. -3.  0. -3.]
 [-3.  0.  0.  0. -3.  0.]]


In [5]:
one_electron_operator_1 = np.zeros((6, 6))
one_electron_operator_1[0,0] = 1
print(one_electron_operator_1)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [8]:
def bereken_elektronenpopulatie(mu_alfa):
    # Hier voer je de berekeningen uit om de elektronenpopulatie op de gewenste site te bepalen
    # Veronderstel dat benzene, H, one_electron_operator, pts, enz. eerder zijn gedefinieerd
    benzene.huckel_matrix = H + mu_alfa * one_electron_operator_1
    benzene.solve_huckel_matrix()
    eigenvectors = benzene.eigenvectors
    D_huckel = pts.making_D(eigenvectors, 6)
    return np.diag(D_huckel)[0]  # Hier krijgen we bijvoorbeeld de elektronenpopulatie op de zesde site

def doel_populatie(mu_alfa, gewenste_populatie):
    # Deze functie geeft het verschil tussen de berekende populatie en de gewenste populatie terug
    berekende_populatie = bereken_elektronenpopulatie(mu_alfa)
    verschil = berekende_populatie**2 - gewenste_populatie**2
    return verschil

def vind_optimale_mu( gewenste_populatie, mu_lower, mu_upper):
    # Bepaal de optimale waarde van mu binnen de opgegeven grenzen
    
    # Pas Brent's methode toe om de optimale waarde van mu te vinden
    optimale_mu = brentq(lambda mu_alfa: doel_populatie(mu_alfa, gewenste_populatie), mu_lower, mu_upper)
    
    return optimale_mu

# Voorbeeldgebruik:
gewenste_populatie = 1.99
mu_lower = -100.0  # Ondergrens voor mu
mu_upper = 100.0  # Bovengrens voor mu

optimale_mu = vind_optimale_mu(gewenste_populatie, mu_lower, mu_upper)
print("Optimale waarde van mu:", optimale_mu)

Optimale waarde van mu: -53.927335145918754


In [11]:
electronpopulation_1 = np.linspace(0.005, 1.995, 3)
print(electronpopulation_1)

[0.005 1.    1.995]


In [33]:
electron_population = [[],[],[],[],[],[]]
S_1_atom = []
S_2_atoms = []
energies = []
potentials = [vind_optimale_mu(gewenste_populatie, mu_lower, mu_upper) for gewenste_populatie in electronpopulation_1]
print(potentials)

for potential in potentials:
    benzene.huckel_matrix = H + potential* one_electron_operator_1
    benzene.solve_huckel_matrix()

    eigenvalues = benzene.eigenvalues 
    eigenvectors = benzene.eigenvectors
    #pts.plot_MOs(eigenvalues, eigenvectors, coordinates, extra_bounds, fig_size = (5,7), unit='Ha')
    D_huckel = pts.making_D(eigenvectors, 6)
    for index, density in enumerate(np.diag(D_huckel)):
        electron_population[index].append(density)

    elec_to_place = 6
    i = 0
    energy = 0
    while elec_to_place >0:
        if elec_to_place > 1:
            elec_in_orbital = 2
        elif elec_to_place == 1:
            elec_in_orbital = 1
        else:
            elec_in_orbital = 0

        energy += elec_in_orbital*(eigenvalues[i]-potential*(eigenvectors.T[i] @ one_electron_operator_1 @ eigenvectors.T[i].T))
        elec_to_place -= elec_in_orbital
        i+=1
    energies.append(energy)

    occ_orbitals = eigenvectors.T[:3].round(6)
    S_ohm_1_atom= []
    S_ohm_2_atoms = []
    for i in range(6):
        atom_i = np.zeros((6,6))
        atom_i[i][i] = 1
        eigvals_i = np.linalg.eigvalsh(occ_orbitals @ atom_i @ occ_orbitals.T).round(6)
        S_ohm_i = 0
        for eigenvalue in eigvals_i:
            if (eigenvalue != 0): 
                S_ohm_i -= (eigenvalue*np.log(eigenvalue) + (1-eigenvalue)*np.log(1-eigenvalue))
        S_ohm_1_atom.append(2*S_ohm_i)
        for j in range(i+1,6):
            atom_j = np.zeros((6,6))
            atom_j[j][j] = 1    
            atom_i_j = atom_i + atom_j
            eigvals_i_j = np.linalg.eigvalsh(occ_orbitals @ atom_i_j @ occ_orbitals.T).round(6)
            S_ohm_i_j = 0
            for eigenvalue in eigvals_i_j:
                if (eigenvalue != 0): 
                    S_ohm_i_j -= (eigenvalue*np.log(eigenvalue) + (1-eigenvalue)*np.log(1-eigenvalue))
            S_ohm_2_atoms.append(2*S_ohm_i_j)
    S_1_atom.append(S_ohm_1_atom)
    S_2_atoms.append(S_ohm_2_atoms)
print(S_1_atom)
print(electron_population[0])
print(S_2_atoms)
print(energies)
print(electron_population)

[76.71190697600056, -3.1675246008161637e-06, -76.71190697600066]
[[0.03495106752068627, 1.2669549058751683, 1.3862820128984774, 1.283340375577896, 1.3862820128984774, 1.2669549058751683], [1.3862943611158904, 1.3862943611198906, 1.3862943611198906, 1.3862943611158904, 1.3862943611198906, 1.3862943611198906], [0.03496304504257068, 1.2669563311756367, 1.3862820269505352, 1.2833390575353212, 1.3862820269505352, 1.2669563311756367]]
[0.005000079864583133, 0.9999999999999998, 1.994999920135417]
[[1.279549983280569, 1.4187144453983673, 1.314562616735744, 1.4187144453983673, 1.279549983280569, 1.1247022870670202, 2.305373414180515, 2.5509272283007727, 2.2590616911730845, 1.9000196414404, 2.7725393290491653, 2.5509272283007727, 1.9000196414404, 2.305373414180515, 1.1247022870670202], [1.8022469813808355, 2.7725887222357812, 2.5460571352748715, 2.7725887222357812, 1.8022469813808355, 1.8022469813808355, 2.7725887222357812, 2.5460557489820106, 2.7725887222357812, 1.8022469813808355, 2.7725887222

In [ ]:
electron_population= []
S_1_atom = []
S_2_atoms = []
energies = []
potentials = [vind_optimale_mu(gewenste_populatie, mu_lower, mu_upper) for gewenste_populatie in electronpopulation_1]
for potential in potentials:
    benzene.huckel_matrix = H + potential* one_electron_operator_1
    benzene.solve_huckel_matrix()

    eigenvalues = benzene.eigenvalues
    eigenvectors = benzene.eigenvectors

    D_huckel = pts.making_D(eigenvectors, 6)
    electron_population.append(np.diag(D_huckel))

        

        
        S_ohm_1_atom= []
        S_ohm_2_atoms = []
        for i in range(6):
            atom_i = np.zeros((6,6))
            atom_i[i][i] = 1
            eigvals_i = np.linalg.eigvalsh(occ_orbitals @ atom_i @ occ_orbitals.T).round(6)
            S_ohm_i = 0
            for eigenvalue in eigvals_i:
                if (eigenvalue != 0): 
                    S_ohm_i -= (eigenvalue*np.log(eigenvalue) + (1-eigenvalue)*np.log(1-eigenvalue))
            S_ohm_1_atom.append(2*S_ohm_i)
            for j in range(i+1,6):
                atom_j = np.zeros((6,6))
                atom_j[j][j] = 1    
                atom_i_j = atom_i + atom_j
                eigvals_i_j = np.linalg.eigvalsh(occ_orbitals @ atom_i_j @ occ_orbitals.T).round(6)
                S_ohm_i_j = 0
                for eigenvalue in eigvals_i_j:
                    if (eigenvalue != 0): 
                        S_ohm_i_j -= (eigenvalue*np.log(eigenvalue) + (1-eigenvalue)*np.log(1-eigenvalue))
                S_ohm_2_atoms.append(2*S_ohm_i_j)
        S_1_atom.append(S_ohm_1_atom)
        S_2_atoms.append(S_ohm_2_atoms)
    electron_population_betas.append(electron_population)
    S_1_atom_betas.append(S_1_atom)
    S_2_atoms_betas.append(S_2_atoms)
    energies_betas.append(energies)
